# 5 monitor robot battery state

change ${CORE_ROOT} to your path of `core`.

In [ ]:
export CORE_ROOT="${HOME}/core"

change ${PJ_ROOT} to your path of `example-guide-demo`.

In [ ]:
export PJ_ROOT="${HOME}/example-guide-demo"
cd ${PJ_ROOT};pwd

example)
```
/Users/user/example-guide-demo
```

## load environment variables

load from `core`

In [ ]:
source ${CORE_ROOT}/docs/environments/azure_aks/env

load from `example-guide-demo`

In [ ]:
source ${PJ_ROOT}/docs/environments/azure_aks/env

## register `cygnus-elasticsearch` as a subscriber of mobile_robot device

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -i -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${MOBILE_ROBOT_SERVICEPATH}" -H "Content-Type: application/json" https://api.${DOMAIN}/orion/v2/subscriptions/ -X POST -d @- <<__EOS__
{
  "subject": {
    "entities": [{
      "idPattern": "${MOBILE_ROBOT_ID}",
      "type": "${MOBILE_ROBOT_TYPE}"
    }],
    "condition": {
      "attrs": ["voltage", "current", "charge", "capacity", "design_capacity", "percentage"]
    }
  },
  "notification": {
    "http": {
      "url": "http://cygnus-elasticsearch:5050/notify"
    },
    "attrs": ["voltage", "current", "charge", "capacity", "design_capacity", "percentage"],
    "attrsFormat": "legacy"
  }
}
__EOS__

example)
```
HTTP/1.1 201 Created
content-length: 0
location: /v2/subscriptions/5cda7d5812c6efcf9ec387fe
fiware-correlator: f2cc0bde-7622-11e9-ba92-0636c4938d99
date: Tue, 14 May 2019 08:33:28 GMT
x-envoy-upstream-service-time: 5
server: envoy
```

In [ ]:
TOKEN=$(cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[0].settings.bearer_tokens[0].token' -r)
curl -sS -H "Authorization: bearer ${TOKEN}" -H "Fiware-Service: ${FIWARE_SERVICE}" -H "Fiware-ServicePath: ${MOBILE_ROBOT_SERVICEPATH}" https://api.${DOMAIN}/orion/v2/subscriptions/ | jq .

example)
```json
[
  {
    "id": "5cd952ef12c6efcf9ec387fd",
    "status": "active",
    "subject": {
      "entities": [
        {
          "idPattern": "rosbot2_01",
          "type": "mobile_robot"
        }
      ],
      "condition": {
        "attrs": [
          "r_mode",
          "r_state",
          "x",
          "y",
          "theta",
          "move_status",
          "move_info",
          "voltage",
          "current",
          "charge",
          "capacity",
          "design_capacity",
          "percentage"
        ]
      }
    },
    "notification": {
      "timesSent": 22009,
      "lastNotification": "2019-05-14T07:57:33.00Z",
      "attrs": [
        "r_mode",
        "r_state",
        "x",
        "y",
        "theta",
        "move_status",
        "move_info",
        "voltage",
        "current",
        "charge",
        "capacity",
        "design_capacity",
        "percentage"
      ],
      "attrsFormat": "legacy",
      "http": {
        "url": "http://cygnus-mongo:5050/notify"
      },
      "lastSuccess": "2019-05-14T07:57:33.00Z",
      "lastSuccessCode": 200
    }
  },
  {
    "id": "5cd959929ea137395f05c133",
    "status": "active",
    "subject": {
      "entities": [
        {
          "idPattern": "rosbot2_01",
          "type": "mobile_robot"
        }
      ],
      "condition": {
        "attrs": [
          "x",
          "y",
          "theta",
          "r_mode"
        ]
      }
    },
    "notification": {
      "timesSent": 21745,
      "lastNotification": "2019-05-14T07:57:33.00Z",
      "attrs": [
        "x",
        "y",
        "theta",
        "r_mode"
      ],
      "attrsFormat": "normalized",
      "http": {
        "url": "http://guide-demo:3000/notify/update-mobilerobot-state/"
      },
      "lastFailure": "2019-05-14T06:45:19.00Z",
      "lastFailureReason": "Server returned nothing (no headers, no data)",
      "lastSuccess": "2019-05-14T07:57:33.00Z",
      "lastSuccessCode": 200
    }
  },
  {
    "id": "5cda7d5812c6efcf9ec387fe",
    "status": "active",
    "subject": {
      "entities": [
        {
          "idPattern": "rosbot2_01",
          "type": "mobile_robot"
        }
      ],
      "condition": {
        "attrs": [
          "voltage",
          "current",
          "charge",
          "capacity",
          "design_capacity",
          "percentage"
        ]
      }
    },
    "notification": {
      "timesSent": 1,
      "lastNotification": "2019-05-14T08:33:28.00Z",
      "attrs": [
        "voltage",
        "current",
        "charge",
        "capacity",
        "design_capacity",
        "percentage"
      ],
      "attrsFormat": "legacy",
      "http": {
        "url": "http://cygnus-elasticsearch:5050/notify"
      },
      "lastSuccess": "2019-05-14T08:33:28.00Z",
      "lastSuccessCode": 200
    }
  }
]
```

## set up Kibana and grafana

### confirm basic auth username & password for Kibana

In [ ]:
cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[]|select(.host == "kibana\\..+$")|.settings.basic_auths[0].username' -r

In [ ]:
cat ${CORE_ROOT}/secrets/auth-tokens.json | jq '.[]|select(.host == "kibana\\..+$")|.settings.basic_auths[0].password' -r

### open kibana

#### for macOS

In [ ]:
open https://kibana.${DOMAIN}/

#### for Ubuntu

In [ ]:
xdg-open https://kibana.${DOMAIN}/

1. show `Management -> Index Patterns`
2. click `Create index pattern`
3. set `cygnus-guide_demo-mobile_robot-*` as Index Pattern, and push `Next step`
4. set `recvTime` as Time Filter field name, and push `Create index pattern`

**Index name is like: `<<IndexPrefix of cygnus>>-<<FIWARE_SERVICE>>-<<FIWARE_SERVICEPATH of depoloyer>>-<<yyyy.mm.dd>>`**

### open grafana

#### for macOS

In [ ]:
open https://grafana.${DOMAIN}/

#### for Ubuntu

In [ ]:
xdg-open https://grafana.${DOMAIN}/

1. add a new Data Source (ElasticSearch)
    * Name: `cygnus-elasticsearch`
    * URL: `http://elasticsearch-logging:9200`
    * Access: `Server(Default)`
    * Index name: `cygnus-guide_demo-mobile_robot-*`
    * Time field name: `recvTime`
    * Version: `6.0+`
2. import `monitoring/dashboard_battery.json`